# Классификация в машинном обучении: практическое руководство

В этом notebook мы разберём четыре фундаментальных алгоритма классификации:
- k-ближайших соседей (k-NN)
- Метод опорных векторов (SVM)
- Деревья решений
- Наивный байес

Для каждого алгоритма покажем:
- Как работает на синтетических данных
- Визуализацию границ решения
- Оценку качества
- Подбор гиперпараметров

In [ ]:
# Импорты
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_moons, load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Алгоритмы
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

# Настройки визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Фиксируем random seed для воспроизводимости
np.random.seed(42)

---
## Часть 1: k-ближайших соседей (k-NN)

### Теория

k-NN — один из самых простых алгоритмов машинного обучения. Идея: чтобы классифицировать новый объект, найдём $k$ ближайших к нему точек из обучающей выборки и выберем наиболее частый класс среди них.

**Ключевые параметры:**
- `n_neighbors` (k): количество соседей
- `metric`: метрика расстояния (euclidean, manhattan, etc.)
- `weights`: веса соседей (uniform или distance)

**Особенности:**
- Требует масштабирования признаков
- Медленное предсказание на больших данных
- Чувствителен к выбору k

In [ ]:
# Создаём синтетические данные
X, y = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                          n_informative=2, n_clusters_per_class=1, 
                          class_sep=1.5, random_state=42)

# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Масштабирование (критически важно для k-NN!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
print(f"Распределение классов: {np.bincount(y_train)}")

In [ ]:
# Функция для визуализации границ решения
def plot_decision_boundary(X, y, model, title, scaler=None):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k', s=50)
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.title(title)
    plt.colorbar()
    plt.show()

In [ ]:
# Обучаем k-NN с разными k
k_values = [1, 3, 5, 15]

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    y_pred = knn.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\nk = {k}")
    print(f"Точность на тестовой выборке: {accuracy:.3f}")
    
    # Визуализация границ решения
    plot_decision_boundary(X_train_scaled, y_train, knn, 
                          f'k-NN (k={k}), accuracy={accuracy:.3f}')

In [ ]:
# Подбор оптимального k с помощью кросс-валидации
k_range = range(1, 31)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    k_scores.append(scores.mean())

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(k_range, k_scores, marker='o')
plt.xlabel('Значение k')
plt.ylabel('Средняя точность (5-fold CV)')
plt.title('Выбор оптимального k (Elbow method)')
plt.axvline(x=k_range[np.argmax(k_scores)], color='r', linestyle='--', 
           label=f'Оптимальное k={k_range[np.argmax(k_scores)]}')
plt.legend()
plt.grid(True)
plt.show()

print(f"Оптимальное k: {k_range[np.argmax(k_scores)]}")
print(f"Максимальная точность: {max(k_scores):.3f}")

### Выводы по k-NN:

- При малом k (1-3): модель переобучается, граница сложная
- При большом k (15+): модель становится более гладкой
- Оптимальное k находится кросс-валидацией
- Масштабирование критически важно!

---
## Часть 2: Метод опорных векторов (SVM)

### Теория

SVM ищет гиперплоскость, которая максимизирует зазор (margin) между классами.

**Ключевые параметры:**
- `C`: параметр регуляризации (большой C = жёсткий штраф за ошибки)
- `kernel`: тип ядра (linear, poly, rbf, sigmoid)
- `gamma`: параметр RBF-ядра (малый = гладкая граница)

**Особенности:**
- Хорошо работает с высокоразмерными данными
- Kernel trick позволяет решать нелинейные задачи
- Требует масштабирования признаков

In [ ]:
# Создаём нелинейно разделимые данные
X_moons, y_moons = make_moons(n_samples=300, noise=0.15, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_moons, y_moons, 
                                                            test_size=0.3, random_state=42)

# Масштабирование
scaler_m = StandardScaler()
X_train_m_scaled = scaler_m.fit_transform(X_train_m)
X_test_m_scaled = scaler_m.transform(X_test_m)

# Визуализация данных
plt.figure(figsize=(10, 6))
plt.scatter(X_train_m[:, 0], X_train_m[:, 1], c=y_train_m, cmap='coolwarm', edgecolors='k')
plt.title('Нелинейно разделимые данные (две луны)')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.show()

In [ ]:
# Сравнение разных ядер SVM
kernels = ['linear', 'poly', 'rbf']

for kernel in kernels:
    if kernel == 'poly':
        svm = SVC(kernel=kernel, degree=3, C=1.0, random_state=42)
    else:
        svm = SVC(kernel=kernel, C=1.0, random_state=42)
    
    svm.fit(X_train_m_scaled, y_train_m)
    y_pred = svm.predict(X_test_m_scaled)
    accuracy = accuracy_score(y_test_m, y_pred)
    
    print(f"\nКерnel: {kernel}")
    print(f"Точность: {accuracy:.3f}")
    print(f"Количество опорных векторов: {len(svm.support_vectors_)}")
    
    # Визуализация
    plot_decision_boundary(X_train_m_scaled, y_train_m, svm, 
                          f'SVM ({kernel} kernel), accuracy={accuracy:.3f}')

In [ ]:
# Влияние параметра C на RBF-ядро
C_values = [0.1, 1.0, 10.0, 100.0]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, C in enumerate(C_values):
    svm = SVC(kernel='rbf', C=C, gamma='scale', random_state=42)
    svm.fit(X_train_m_scaled, y_train_m)
    
    accuracy = accuracy_score(y_test_m, svm.predict(X_test_m_scaled))
    
    # Визуализация границ
    h = 0.02
    x_min, x_max = X_train_m_scaled[:, 0].min() - 1, X_train_m_scaled[:, 0].max() + 1
    y_min, y_max = X_train_m_scaled[:, 1].min() - 1, X_train_m_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    axes[idx].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    axes[idx].scatter(X_train_m_scaled[:, 0], X_train_m_scaled[:, 1], 
                     c=y_train_m, cmap='coolwarm', edgecolors='k')
    axes[idx].set_title(f'C={C}, accuracy={accuracy:.3f}\nОпорных векторов: {len(svm.support_vectors_)}')
    axes[idx].set_xlabel('Признак 1')
    axes[idx].set_ylabel('Признак 2')

plt.tight_layout()
plt.show()

In [ ]:
# Grid Search для подбора гиперпараметров
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'kernel': ['rbf']
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_m_scaled, y_train_m)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучшая точность (CV): {grid_search.best_score_:.3f}")

# Оценка на тестовой выборке
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test_m_scaled)
print(f"Точность на тесте: {accuracy_score(y_test_m, y_pred_best):.3f}")

### Выводы по SVM:

- Линейное ядро плохо работает на нелинейных данных
- RBF-ядро универсально и часто лучший выбор
- Малый C: гладкая граница, риск недообучения
- Большой C: сложная граница, риск переобучения
- Grid Search критически важен для хороших результатов

---
## Часть 3: Деревья решений

### Теория

Дерево решений — последовательность условий (if-then), которые разделяют данные на классы.

**Ключевые параметры:**
- `max_depth`: максимальная глубина дерева
- `min_samples_split`: минимум примеров для разбиения узла
- `min_samples_leaf`: минимум примеров в листовом узле
- `criterion`: критерий разбиения (gini или entropy)

**Особенности:**
- Очень интерпретируемы
- Не требуют масштабирования
- Склонны к переобучению без ограничений

In [ ]:
# Используем датасет Iris
iris = load_iris()
X_iris, y_iris = iris.data[:, :2], iris.target  # берём только первые 2 признака для визуализации

# Оставим только 2 класса для упрощения
mask = y_iris != 2
X_iris, y_iris = X_iris[mask], y_iris[mask]

X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42)

print(f"Признаки: {iris.feature_names[:2]}")
print(f"Классы: {[iris.target_names[i] for i in [0, 1]]}")

In [ ]:
# Деревья с разной глубиной
depths = [1, 2, 3, 10]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, depth in enumerate(depths):
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    tree.fit(X_train_iris, y_train_iris)
    
    accuracy = accuracy_score(y_test_iris, tree.predict(X_test_iris))
    
    # Визуализация границ
    h = 0.02
    x_min, x_max = X_train_iris[:, 0].min() - 0.5, X_train_iris[:, 0].max() + 0.5
    y_min, y_max = X_train_iris[:, 1].min() - 0.5, X_train_iris[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = tree.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    axes[idx].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    axes[idx].scatter(X_train_iris[:, 0], X_train_iris[:, 1], 
                     c=y_train_iris, cmap='coolwarm', edgecolors='k')
    axes[idx].set_title(f'max_depth={depth}, accuracy={accuracy:.3f}')
    axes[idx].set_xlabel(iris.feature_names[0])
    axes[idx].set_ylabel(iris.feature_names[1])

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация самого дерева
tree_best = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_best.fit(X_train_iris, y_train_iris)

plt.figure(figsize=(20, 10))
plot_tree(tree_best, feature_names=iris.feature_names[:2], 
         class_names=[iris.target_names[i] for i in [0, 1]], 
         filled=True, rounded=True, fontsize=12)
plt.title('Визуализация дерева решений (max_depth=3)')
plt.show()

In [ ]:
# Вычисление Gini impurity вручную
def gini_impurity(y):
    """Вычисляет Gini impurity для массива меток"""
    classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    gini = 1 - np.sum(probabilities ** 2)
    return gini

# Пример
print("Примеры вычисления Gini impurity:\n")
print(f"Чистый узел [0,0,0,0,0]: Gini = {gini_impurity([0,0,0,0,0]):.3f}")
print(f"Сбалансированный [0,0,1,1]: Gini = {gini_impurity([0,0,1,1]):.3f}")
print(f"Несбалансированный [0,0,0,1]: Gini = {gini_impurity([0,0,0,1]):.3f}")
print(f"\nGini всей выборки: {gini_impurity(y_train_iris):.3f}")

In [ ]:
# Важность признаков
tree_full = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_full.fit(X_train_iris, y_train_iris)

importances = tree_full.feature_importances_
feature_names = iris.feature_names[:2]

plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances)
plt.xlabel('Важность признака')
plt.title('Feature Importance в дереве решений')
plt.show()

for name, imp in zip(feature_names, importances):
    print(f"{name}: {imp:.3f}")

### Выводы по деревьям решений:

- max_depth=1: слишком простая модель (недообучение)
- max_depth=10: переобучение, сложные границы
- max_depth=2-3: оптимальный баланс
- Дерево легко интерпретировать
- Feature importance показывает важность признаков

---
## Часть 4: Наивный байес и текстовая классификация

### Теория

Наивный байес использует теорему Байеса для классификации. Особенно эффективен для текстовой классификации.

**Варианты:**
- `MultinomialNB`: для подсчёта частот (текст)
- `GaussianNB`: для непрерывных признаков
- `BernoulliNB`: для бинарных признаков

**Особенности:**
- Очень быстрый
- Хорош для текстов
- Предполагает независимость признаков (наивное предположение)

In [ ]:
# Простой пример: классификация коротких текстов
texts = [
    "I love this movie",
    "This film is great",
    "Amazing movie, best ever",
    "Wonderful film experience",
    "I hate this movie",
    "Terrible film, worst ever",
    "Bad movie, waste of time",
    "Awful film, very boring"
]

labels = [1, 1, 1, 1, 0, 0, 0, 0]  # 1 = положительный, 0 = отрицательный

# Векторизация текста (Count Vectorization)
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(texts)

print(f"Словарь: {vectorizer.get_feature_names_out()}")
print("\nМатрица признаков (разреженная):")
print(X_text.toarray())

In [ ]:
# Обучение Naive Bayes
nb = MultinomialNB()
nb.fit(X_text, labels)

# Предсказания
test_texts = [
    "I love this film",
    "This movie is terrible",
    "Great experience"
]

X_test_text = vectorizer.transform(test_texts)
predictions = nb.predict(X_test_text)
probs = nb.predict_proba(X_test_text)

print("Предсказания:\n")
for text, pred, prob in zip(test_texts, predictions, probs):
    sentiment = "Положительный" if pred == 1 else "Отрицательный"
    print(f"Текст: '{text}'")
    print(f"Класс: {sentiment}")
    print(f"Вероятности: Отриц={prob[0]:.3f}, Полож={prob[1]:.3f}\n")

In [ ]:
# Вероятности слов в каждом классе
feature_names = vectorizer.get_feature_names_out()
log_probs_neg = nb.feature_log_prob_[0]
log_probs_pos = nb.feature_log_prob_[1]

print("Наиболее вероятные слова для каждого класса:\n")

# Топ-5 слов для отрицательного класса
top_neg = np.argsort(log_probs_neg)[-5:]
print("Отрицательный класс:")
for idx in top_neg:
    print(f"  {feature_names[idx]}: {np.exp(log_probs_neg[idx]):.3f}")

# Топ-5 слов для положительного класса
top_pos = np.argsort(log_probs_pos)[-5:]
print("\nПоложительный класс:")
for idx in top_pos:
    print(f"  {feature_names[idx]}: {np.exp(log_probs_pos[idx]):.3f}")

In [ ]:
# Gaussian Naive Bayes на числовых данных (Iris)
gnb = GaussianNB()
gnb.fit(X_train_iris, y_train_iris)

y_pred_gnb = gnb.predict(X_test_iris)
accuracy_gnb = accuracy_score(y_test_iris, y_pred_gnb)

print("Gaussian Naive Bayes на Iris:")
print(f"Точность: {accuracy_gnb:.3f}")

# Confusion Matrix
cm = confusion_matrix(y_test_iris, y_pred_gnb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=[iris.target_names[i] for i in [0,1]], 
           yticklabels=[iris.target_names[i] for i in [0,1]])
plt.title('Confusion Matrix для Gaussian NB')
plt.ylabel('Истинный класс')
plt.xlabel('Предсказанный класс')
plt.show()

### Выводы по Naive Bayes:

- Очень быстрый и простой алгоритм
- MultinomialNB отлично подходит для текстов
- GaussianNB для непрерывных признаков
- Можно интерпретировать вероятности слов
- Несмотря на наивное предположение, работает хорошо

---
## Часть 5: Сравнение всех алгоритмов

Давайте сравним все 4 алгоритма на одном датасете.

In [ ]:
# Используем исходные данные
models = {
    'k-NN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'SVM (RBF)': SVC(kernel='rbf', C=1.0, gamma='scale'),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Gaussian NB': GaussianNB()
}

results = []

for name, model in models.items():
    # k-NN и SVM требуют масштабирования
    if name in ['k-NN (k=5)', 'SVM (RBF)']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    results.append({'Алгоритм': name, 'Точность': accuracy})
    
    print(f"\n{name}:")
    print(f"Точность: {accuracy:.3f}")
    print(classification_report(y_test, y_pred, target_names=['Класс 0', 'Класс 1']))

# Визуализация сравнения
results_df = pd.DataFrame(results)
plt.figure(figsize=(10, 6))
plt.barh(results_df['Алгоритм'], results_df['Точность'])
plt.xlabel('Точность')
plt.title('Сравнение алгоритмов классификации')
plt.xlim(0, 1)
for i, v in enumerate(results_df['Точность']):
    plt.text(v + 0.01, i, f"{v:.3f}", va='center')
plt.show()

---
## Итоговые выводы

### Когда использовать каждый алгоритм:

**k-NN:**
- ✅ Малые датасеты
- ✅ Простая интерпретация
- ✅ Локальные зависимости
- ❌ Большие данные (медленно)
- ❌ Высокоразмерные данные

**SVM:**
- ✅ Высокоразмерные данные
- ✅ Чёткое разделение классов
- ✅ Нелинейные задачи (RBF kernel)
- ❌ Большие датасеты (медленное обучение)
- ❌ Трудная интерпретация

**Decision Trees:**
- ✅ Интерпретируемость
- ✅ Категориальные признаки
- ✅ Нелинейные взаимодействия
- ❌ Склонность к переобучению
- ❌ Нестабильность (малые изменения данных → разные деревья)

**Naive Bayes:**
- ✅ Текстовая классификация
- ✅ Очень быстрый
- ✅ Малые данные
- ❌ Предположение независимости
- ❌ Непрерывные признаки с корреляциями

### Практические советы:

1. **Всегда начинайте с простого**: baseline модель (Naive Bayes или Decision Tree)
2. **Масштабируйте данные** для k-NN и SVM
3. **Используйте кросс-валидацию** для оценки
4. **Grid Search** для подбора гиперпараметров
6. **Смотрите на разные метрики**: не только accuracy, но и precision, recall, F1
7. **Визуализируйте** границы решения и confusion matrix